In [1]:
import subprocess, os, time, re, csv, sys, io

import pandas as pd
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt

from pathlib import Path
from typing import List, Tuple
from collections import defaultdict

from visualization import *

root_dir = "/valiant02/masi/zuol1/data/cibs_brain2/"
dest_root_dir = "/nfs2/harmonization/BIDS/CIBS_BRAIN2_Harmonized"
demo_metadata_csv = "labels/demo_metadata.csv"
result_dir = "dataset_cache"

In [3]:
def dump(path_list, outfile):
    with open(outfile, "w") as f:
        for p in sorted(path_list):
            f.write(str(p) + "\n")
    print(f"Cached {len(path_list)} paths to '{outfile}'")


def load_paths_from_file(file_path: str) -> List[str]:
    """Load paths from a file."""
    with open(file_path, "r") as f:
        return [line.strip() for line in f if line.strip()]

In [5]:
lines = []
for dirpath, dirnames, filenames in os.walk(Path(root_dir)):
    if dirnames:
        continue
    file_type = Path(dirpath).name
    session = Path(dirpath).parent.name
    if session not in {"00", "12"}:
        continue
    for fname in filenames:
        if fname.endswith(".nii") or fname.endswith(".nii.gz"):
            lines.append(str(Path(dirpath) / fname))
dump(lines, f"{result_dir}/nifti_all.txt")

Cached 3730 paths to 'dataset_cache/nifti_all.txt'


In [3]:
lines = load_paths_from_file(f"{result_dir}/nifti_nii.txt")
print(f"Loaded {len(lines)} paths from nifti_nii.txt")

Loaded 722 paths from nifti_nii.txt


In [4]:
depths = [len(Path(p).parts) for p in lines]

# Check whether the depths of all addresses are same
print(len(set(depths)) == 1)

True


In [6]:
subject_sessions = defaultdict(set)

for fp in lines:
    p = Path(fp)
    if len(p.parents) < 3:
        continue
    session = p.parents[1].name
    file_type = p.parents[0].name
    if session not in ["00", "12"] or file_type != "nii":
        continue
    subject_dir = p.parents[2]
    subject_sessions[subject_dir].add(session)

only_00 = [p for p, s in subject_sessions.items() if s == {"00"}]
both_00_12 = [p for p, s in subject_sessions.items() if s == {"00", "12"}]
only_12 = [p for p, s in subject_sessions.items() if s == {"12"}]

dump(only_00, f"{result_dir}/subject_only_00.txt")
dump(both_00_12, f"{result_dir}/subject_00_and_12.txt")
dump(only_12, f"{result_dir}/subject_only_12.txt")

print(f"Only baseline data (00 only): {len(only_00)}")
print(f"Valid data (00 + 12): {len(both_00_12)}")
print(f"Only 12-month data: {len(only_12)}")

Cached 76 paths to 'dataset_cache/subject_only_00.txt'
Cached 37 paths to 'dataset_cache/subject_00_and_12.txt'
Cached 22 paths to 'dataset_cache/subject_only_12.txt'
Only baseline data (00 only): 76
Valid data (00 + 12): 37
Only 12-month data: 22


In [ ]:
file_pat = re.compile(r"BRAIN_T1-3D.*_n4_reg_harmonized_harmonized_fusion\.nii\.gz$")

good_paths: list[Path] = []
bad_subjects: list[str] = []
subject_roots = only_00 + both_00_12 + only_12

for subj_root in subject_roots:
    subj_root = Path(subj_root)
    subj_id = subj_root.name

    for ses in ("00", "12"):
        ses_dir = subj_root / ses
        if not ses_dir.is_dir():
            continue
        proc_dir = ses_dir / "proc"
        if not proc_dir.is_dir():
            bad_subjects.append(f"{subj_id}: missing directory {ses}/proc")
            break
        matches = [
            p for p in proc_dir.iterdir() if p.is_file() and file_pat.search(p.name)
        ]
        if not matches:
            bad_subjects.append(
                f"{subj_id}: no BRAIN_T1-3D harmonized file in {ses}/proc"
            )
            break
        good_paths.extend(matches)

dump(good_paths, f"{result_dir}/harmonized_all.txt")

if bad_subjects:
    print("\n Subjects with missing data:")
    for msg in bad_subjects:
        print("  -", msg)
else:
    print("\n All subjects passed the file-check.")

Cached 168 paths to 'dataset_cache/harmonized_all.txt'

 Subjects with missing data:
  - BRA105: no BRAIN_T1-3D harmonized file in 00/proc
  - BRA107: no BRAIN_T1-3D harmonized file in 12/proc
  - BRA132: no BRAIN_T1-3D harmonized file in 12/proc
  - VUM241: no BRAIN_T1-3D harmonized file in 12/proc


In [11]:
good_paths = load_paths_from_file(f"{result_dir}/harmonized_all.txt")
print(f"Loaded {len(good_paths)} paths from harmonized_all.txt")


def extract_modality(fname: str) -> str:
    stem = fname
    for ext in (".nii.gz", ".nii"):
        if stem.endswith(ext):
            stem = stem[: -len(ext)]
            break
    try:
        seq_part = stem.split("_BRAIN_")[1]
    except IndexError:
        return ""
    tokens = seq_part.split("-")
    return "-".join(tokens[:2]) if len(tokens) >= 2 else ""


def parse_path(file_path: str) -> dict | None:
    p = Path(file_path)
    parts = p.parts
    if len(parts) != 10:
        raise RuntimeError(f"File address not valid.\nAddress: {file_path}")
    subject = parts[6]
    session = parts[7]
    scan_type = extract_modality(parts[9])
    if scan_type not in {"T1-3D"}:
        return None
    return {
        "filepath": p.resolve(),
        "scan_type": scan_type,
        "subject_id": f"sub-{subject}",
        "session_id": f"ses-{session}",
    }


rows = [row for fp in good_paths if (row := parse_path(fp))]
df = pd.DataFrame(rows)

Loaded 168 paths from harmonized_all.txt


In [12]:
df["run"] = (
    df.groupby(["scan_type", "subject_id", "session_id"])
    .cumcount()
    .add(1)
    .astype("string")
)

mask = (
    df.groupby(["scan_type", "subject_id", "session_id"])["run"].transform("size").eq(1)
)
df.loc[mask, "run"] = ""

In [13]:
def make_links(row):
    prefix = f"{row.subject_id}_{row.session_id}"
    if row.run:
        prefix += f"_run-{row.run}"
    if row.scan_type == "T1-3D":
        fname = f"{prefix}_T1w"
        subdir = "anat"
    else:
        raise ValueError(f"Unknown scan type: {row.scan_type}")
    base = f"{dest_root_dir}/{row.subject_id}/{row.session_id}/{subdir}/{fname}"
    row["nii_link"] = base + ".nii.gz"
    return row


df = df.apply(make_links, axis=1)

In [14]:
df.sort_values(["scan_type", "subject_id", "session_id", "run"], inplace=True)

In [15]:
mask = (
    (df["scan_type"] == "T1-3D")
    & df["run"].notna()
    & (df["run"] != "")
    # & (df["run"] == "")
)

if mask.any():
    print(df.loc[mask])
else:
    print("Nothing here.")

Nothing here.


In [16]:
df.to_csv(f"{result_dir}/data.tsv", sep="\t")

In [17]:
def load_dataframe(path: str) -> pd.DataFrame:
    if path.endswith(".pkl"):
        return pd.read_pickle(path)
    return pd.read_csv(path, sep=None, engine="python")


def _collect_link_pairs(df: pd.DataFrame) -> List[Tuple[str, str]]:
    colmap = {
        "filepath": "nii_link",
    }

    present_pairs = [
        (src, dst)
        for src, dst in colmap.items()
        if src in df.columns and dst in df.columns
    ]
    if not present_pairs:
        raise ValueError("No recognised source/target column pairs found.")

    commands = []
    for _, row in df.iterrows():
        for src, dst in present_pairs:
            s, d = row[src], row[dst]
            if pd.notna(s) and pd.notna(d) and str(s).strip() and str(d).strip():
                commands.append(f'mkdir -p "{Path(d).parent}"')
                commands.append(f'ln -s "{s}" "{d}"')
    return commands


def write_link_commands(
    df: pd.DataFrame,
    output_txt: str | Path,
    *,
    overwrite: bool = True,
) -> None:
    output_txt = Path(output_txt)
    mode = "w" if overwrite else "a"

    commands = _collect_link_pairs(df)

    with output_txt.open(mode, encoding="utf-8") as f:
        for cmd in commands:
            f.write(cmd + "\n")

    print(f"{len(commands)} link commands written to {output_txt.resolve()}")

In [18]:
df = load_dataframe(f"{result_dir}/data.tsv")

In [19]:
write_link_commands(df, f"{result_dir}/data_link_command.txt")

336 link commands written to /nfs/ForHenry/brain_ventricle/dataset_cache/data_link_command.txt


In [24]:
vum_meta_df = pd.read_csv("labels/vum_metadata.csv", dtype={"SessionID": str})

only_00 = load_paths_from_file(f"{result_dir}/subject_only_00.txt")
both_00_12 = load_paths_from_file(f"{result_dir}/subject_00_and_12.txt")
only_12 = load_paths_from_file(f"{result_dir}/subject_only_12.txt")

checks = [
    ("only_00", only_00, ["00"]),
    ("both_00_12", both_00_12, ["00", "12"]),
    ("only_12", only_12, ["12"]),
]

for label, subject_list, sessions in checks:
    for subject_addr in subject_list:
        subject_id = Path(subject_addr).name
        if not subject_id.startswith("VUM"):
            continue
        for ses in sessions:
            mask = (vum_meta_df["VUM_ID"] == subject_id) & (
                vum_meta_df["SessionID"] == ses
            )
            if not mask.any():
                print(f"Missing metadata for {subject_id} session {ses}")